In [ ]:
import geopandas as gpd
import folium
import mapclassify as mc
from shapely.geometry import LineString, MultiLineString
import numpy as np

In [ ]:
# criterio lineas o ramales
criterio = 'lineas'

In [ ]:
# Leer las grillas y las lineas nacionales
grilla = gpd.read_file('../carto/grilla.zip')

lineas = gpd.read_file('../carto/lineas-nacionales.zip')
lineas = lineas.loc[lineas.SENTIDO == 'IDA',['ID','LINEA','RAMAL','geometry']]
#lineas.to_file('../carto/lineas_nacionales.geojson',driver='GeoJSON')

In [ ]:
conteo_x_grilla = gpd.sjoin(grilla,lineas,how='inner',predicate='intersects')
conteo_x_grilla = conteo_x_grilla.reindex(columns = ['id','geometry','LINEA'])

if criterio == 'lineas':
    conteo_x_grilla = conteo_x_grilla.drop_duplicates(subset=['id','LINEA'])
    
conteo_x_grilla = conteo_x_grilla.groupby('id',as_index=False).size()
conteo_x_grilla = grilla.merge(conteo_x_grilla)

# Obtener para cada grilla cuantos ramales-lineas pasan
conteo_x_grilla = conteo_x_grilla.rename(columns = {'size':'conteo'})

conteo_x_grilla.to_file(f'../carto/{criterio}_x_grilla.geojson',driver='GeoJSON')


In [ ]:
# Agrupar las grillas en cantidad de gramales segun Fisher Jenks
fj = mc.FisherJenks(conteo_x_grilla.conteo)
bins = fj.bins
bins = [0] + list(bins)
bins[-1] = bins[-1]+1


In [ ]:
m = folium.Map(location=[-34.55412,-58.41287], zoom_start=11,tiles='cartodbpositron')

def style_function(feature):
    return {
        "weight": 1,
        "opacity": 0.4,
        "color": 'red',
    }

folium.GeoJson('../carto/lineas_nacionales.geojson',
               name="Lineas Nacionales",
               style_function= style_function).add_to(m)

    
c = folium.Choropleth(
    geo_data=conteo_x_grilla,
    name=f"Cantidad de {criterio} por grilla",
    data=conteo_x_grilla,
    columns=["id", "conteo"],
    key_on="feature.properties.id",
    fill_color="Greys",
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name=f"{criterio} por grilla",
    bins = bins,
    reset=True,
).add_to(m)

folium.LayerControl().add_to(m)
if criterio == 'lineas':
    m.save('../index.html')
m.save(f'../{criterio}.html')
m